# Unused for now but maybe useful for later

## Apparently there is a Latex magic function in Jupyter Notebook - Cool!!!

In [3]:
%%latex
$$
\frac{\sum_{i=1}^{n=256} \bar{U_i}}{\sum_{i=1}^{n=256} c_i\bar{U_i}} \ge \alpha
$$

<IPython.core.display.Latex object>

## Convert all nominal columns from dtype Object to dtype category

In [ ]:
def nominal_columns_to_categorical():
    object_type_columns = raw_data.dtypes.values == 'O'
    category_df = raw_data.loc[:, object_type_columns]
    
    for column in category_df.columns:
        category_df.loc[:, column] = category_df[column].astype('category')

    categories = []
    for column in category_df.columns:
        c = category_df.loc[:, column].cat.categories
        categories.extend(c.values)

    return categories, category_df

In [ ]:
categories, category_df = nominal_columns_to_categorical()

### Convert categorical to integer using Scikit-Learn 

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [ ]:
label_encoder.fit(categories)

In [ ]:
for column in category.columns:
    category_df.loc[:, column] = category_df.loc[:, column].map(lambda x : label_encoder.transform(x))

In [ ]:
category_df.head()

## Histogram

In [ ]:
# this function plots 2 histograms against the savings
def create_histograms_by_savings(df, column, xlabel, xlim, ylim, bins):
    ax0, ax1 = df.hist(column=column, by='savings', figsize=(14,2.5), 
                       bins=bins, alpha=0.7, xlabelsize=16, ylabelsize=16);
    for ax in [ax0, ax1]:
        ax.set_xlabel(xlabel, fontsize=20);
        ax.set_ylabel('Count', fontsize=20)
        ax.set_yscale('log')
        ax.set_ylim(ylim)
        ax.set_xlim(xlim)
    
    ax0.set_title('Savings < 50,000', fontsize=24);
    ax1.set_title('Savings > 50,000', fontsize=24);


## Split up all nominal columns into dummy variables

In [ ]:
def nominal_columns_to_dummy():
    nominal_columns = [c[0] for c in columns if c[1] == 'nominal']
    dummy_prefix = [c[2] for c in columns if c[1] == 'nominal']
    
    dummy_columns = [pd.get_dummies(raw_data[col], prefix=pref) for col, pref in zip(nominal_columns, dummy_prefix)]
    
    data = pd.concat(dummy_columns, axis=1)
    print 'the number of columns is {:d}'.format(data.shape[1])
    
    # check size
    count_distinct_values = 0
    for column in nominal_columns:
        count_distinct_values += len(raw_data[column].unique())
    
    assert count_distinct_values == data.shape[1], "mismatch between number of dummy columns and unique values"
    
    return data
    
    

In [ ]:
data = nominal_columns_to_dummy()


## Feature Importance from Random Forest Classifier


In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [ ]:
# generate cross-validation data set
class RandomForest(object):
    
    def __init__(self, X, y, description=''):
        
        self.description = description
        print '\n*** {:s} ***\n'.format(self.description)
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.3, random_state=0)
        
        print 'size of training data: {:7d}, {:3d}'.format(self.X_train.shape[0], self.X_train.shape[1])
        print 'size of test data:     {:7d}, {:3d}'.format(self.X_test.shape[0], self.X_test.shape[1])
        
        self.rf_clf = RandomForestClassifier(n_estimators = 100)
        self.rf_clf.fit(self.X_train.values, self.y_train.values)
        
        self.y_pred = self.rf_clf.predict(self.X_test)
        self.accuracy = accuracy_score(self.y_test, self.y_pred)
        print 'accuracy = {:.4f}'.format(self.accuracy)
        print 'class imbalance'
        n_test = self.y_test.shape[0]
        n_test_class = self.y_test.sum()
        n_train = self.y_train.shape[0]
        n_train_class = self.y_train.sum()
        print '\ttest set :{:7d},{:5d} : {:.4f}'.format(n_test, n_test_class, 
                                               (n_test - n_test_class)/float(n_test)) 
        print '\ttrain set:{:7d},{:5d} : {:.4f}'.format(n_train, n_train_class, 
                                               (n_train - n_train_class)/float(n_train))
        self.importances = self.rf_clf.feature_importances_



In [ ]:
y0 = data.loc[:,'savings']
X0 = data.drop('savings', axis=1)
rf = RandomForest(X0, y0, 'basic setup, all columns, no feature engineering')


## Correlations

In [ ]:
corr = data.corr()


In [ ]:
def find_strong_correlations(threshold):
    c = corr.unstack()
    z = c.sort_values()
    correlated = (z > threshold) & (z < 1.0)
    uncorrelated = (z < -threshold) & (z > -1.0)
    correlations = z[correlated | uncorrelated].drop_duplicates()
    return correlations



In [ ]:
correlations = find_strong_correlations(0.8)

In [ ]:
def find_correlated_columns(column):
    result = []
    for c,v in zip(correlations.index, correlations.values):
        if column in c[0]:
            result.append((c[1],v))
        if column in c[1]:
            result.append((c[0], v))
    return result


In [ ]:
find_correlated_columns('year')

In [ ]:
x = find_strong_correlations(0.70)
print x

In [ ]:
# plot correlation plot
fig, ax = plt.subplots(figsize=(12,12));
cax = ax.matshow(corr, cmap=plt.cm.coolwarm);
cbar = plt.colorbar(cax, fraction=0.046, pad=0.04);
cbar.set_label('Correlation',size=20)
cbar.ax.tick_params(labelsize=16)
plt.yticks(range(corr.columns.values.shape[0]), fontsize=12);
ax.set_yticklabels(corr.columns.values);
plt.xticks(fontsize=16);
ax.xaxis.set_ticks_position('bottom')
